In [141]:
import pandas as pd

df_y = pd.read_csv("~/MyWork/BrokerConnector/historical_data/ZION_mintue.csv")
df_x = pd.read_csv("~/MyWork/BrokerConnector/historical_data/LNC_mintue.csv")




dates = df_y.tradingDay.unique()
date= dates[-1]
def getIndexToTimestampDf(df,date):
    newDf = pd.DataFrame(df)
    newDf=newDf.loc[newDf['tradingDay']== date]
    
    #print(len(newDf))
    
    newDf['timestamp'] = pd.to_datetime(newDf['timestamp'])
    newDf.set_index('timestamp', inplace = True)
    
    return newDf

#print (getIndexToTimestampDf(df_y, date))

lenDiff=len(df_y) - len(df_x)
if (lenDiff!=0):
    print("data len is diff "+ str(lenDiff))
    print(df_y.head(1))
    print(df_x.head(1))

data len is diff 1233
  symbol                  timestamp  tradingDay      open      high       low  \
0   ZION  2017-10-09T09:30:00-04:00  2017-10-09  47.80106  47.80106  47.55194   

    close  volume  
0  47.647    8292  
  symbol                  timestamp  tradingDay   open   high    low  close  \
0    LNC  2017-10-09T09:30:00-04:00  2017-10-09  75.13  75.37  75.13  75.13   

   volume  
0    5464  


In [170]:
# fill missing minutes in df 


data_y = getIndexToTimestampDf(df_y,date).resample('1min').interpolate(method='linear')
data_x = getIndexToTimestampDf(df_x,date).resample('1min').interpolate(method='linear')

if (len(data_x)< len(data_y)):
    diff = len(data_x)- len(data_y)
    data_y= data_y.drop(data_y.index[diff])
    print("drop y rows last: ", diff)
elif (len(data_x)> len(data_y)):
    diff = len(data_y)- len(data_x)
    data_x= data_x.drop(data_x.index[diff])
    print("drop x rows last: ", diff)

360
355
drop y rows last:  -1


In [253]:
# function test cointergration
import statsmodels.tsa.stattools as ts 

def isCoint(df_y, df_x, test_window=60):
    ts_y = (df_y['high']+df_y['low'])/2
    ts_x = (df_x['high']+df_x['low'])/2
    
    
    for i in range (0, len(ts_y)-test_window):   
        #print(i,i+test_window)
        result= ts.coint(ts_y[i:i+test_window],ts_x[i:i+test_window])
        if ( result[1]< 0.01 ): #pvalue,  If the pvalue is small, below a critical size, then we can reject the hypothesis that there is no cointegrating relationship.
            print("coin_test is significant",result, ts_y.index[i],ts_y.index[i+test_window])
        else:
            pass
        
    
    print(len(ts_y),len(ts_x))

test_windows=[10,20,30,40,50,60]
for i in test_windows:
    print(" ==== coint test window %i min"%i)
    isCoint(data_y,data_x,test_window=i)

 ==== coint test window 10 min


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/statsmodels/regression/linear_model.py:1353: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/statsmodels/regression/linear_model.py:1353: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


390 390
 ==== coint test window 20 min
390 390
 ==== coint test window 30 min
coin_test is significant (-3.9668080211537773, 0.0080034184303838934, array([-4.31395736, -3.55493606, -3.19393252])) 2018-01-05 16:48:00 2018-01-05 17:18:00
coin_test is significant (-4.3051098616671606, 0.0025138548083582864, array([-4.31395736, -3.55493606, -3.19393252])) 2018-01-05 16:56:00 2018-01-05 17:26:00
coin_test is significant (-5.0511961559365819, 0.00013241382941945915, array([-4.31395736, -3.55493606, -3.19393252])) 2018-01-05 16:57:00 2018-01-05 17:27:00
coin_test is significant (-4.4052793727468345, 0.0017450819382920999, array([-4.31395736, -3.55493606, -3.19393252])) 2018-01-05 16:58:00 2018-01-05 17:28:00
390 390
 ==== coint test window 40 min
390 390
 ==== coint test window 50 min
390 390
 ==== coint test window 60 min
coin_test is significant (-4.1131086015298326, 0.0049208323860880374, array([-4.09169685, -3.44165109, -3.11711613])) 2018-01-05 14:43:00 2018-01-05 15:43:00
coin_test is s

In [252]:
#ols regression 
import statsmodels.api as sm

dfx=(data_x['high']+data_x['low'])/2
dfy=(data_y['high']+data_y['low'])/2

window=30
for n in range (window, len(dfx)):
    x=dfx[n-window:n]
    y=dfy[n-window:n]
    
    #print(n-window, x.index[0],x.index[29])
    #print(x)
    print("---")
    
    x=sm.add_constant(x)
    model = sm.OLS(y,x).fit()
    print(x.index[29])
    print(model.params)

---
2018-01-05 14:59:00
const    16.513536
0         0.436706
dtype: float64
---
2018-01-05 15:00:00
const    15.404488
0         0.450767
dtype: float64
---
2018-01-05 15:01:00
const    18.396875
0         0.412791
dtype: float64
---
2018-01-05 15:02:00
const    19.591343
0         0.397610
dtype: float64
---
2018-01-05 15:03:00
const    19.021521
0         0.404794
dtype: float64
---
2018-01-05 15:04:00
const    19.021226
0         0.404748
dtype: float64
---
2018-01-05 15:05:00
const    18.543734
0         0.410744
dtype: float64
---
2018-01-05 15:06:00
const    18.493535
0         0.411340
dtype: float64
---
2018-01-05 15:07:00
const    18.936834
0         0.405695
dtype: float64
---
2018-01-05 15:08:00
const    20.528290
0         0.385531
dtype: float64
---
2018-01-05 15:09:00
const    23.025841
0         0.353867
dtype: float64
---
2018-01-05 15:10:00
const    24.394277
0         0.336525
dtype: float64
---
2018-01-05 15:11:00
const    21.568285
0         0.372328
dtype: float64

In [222]:
#pd rolling ols

#from pandas.stats.api import ols
x=(data_x['high']+data_x['low'])/2
y=(data_y['high']+data_y['low'])/2


print(dfx)

timestamp
2018-01-05 14:30:00    79.20250
2018-01-05 14:31:00    79.07000
2018-01-05 14:32:00    79.00500
2018-01-05 14:33:00    78.88500
2018-01-05 14:34:00    78.88500
2018-01-05 14:35:00    78.88000
2018-01-05 14:36:00    78.90000
2018-01-05 14:37:00    78.94500
2018-01-05 14:38:00    78.90500
2018-01-05 14:39:00    78.72550
2018-01-05 14:40:00    78.77750
2018-01-05 14:41:00    78.98000
2018-01-05 14:42:00    79.07000
2018-01-05 14:43:00    79.07250
2018-01-05 14:44:00    79.07500
2018-01-05 14:45:00    79.18750
2018-01-05 14:46:00    79.20000
2018-01-05 14:47:00    78.98000
2018-01-05 14:48:00    78.91875
2018-01-05 14:49:00    78.85750
2018-01-05 14:50:00    78.93000
2018-01-05 14:51:00    78.86500
2018-01-05 14:52:00    78.86000
2018-01-05 14:53:00    78.85500
2018-01-05 14:54:00    78.85000
2018-01-05 14:55:00    78.88750
2018-01-05 14:56:00    78.87375
2018-01-05 14:57:00    78.86000
2018-01-05 14:58:00    78.80500
2018-01-05 14:59:00    78.75500
                         ...  

In [5]:
#trading date range 

date=df_y.tradingDay.unique()
date=df_x.tradingDay.unique()
date

array(['2017-10-09', '2017-10-10', '2017-10-11', '2017-10-12',
       '2017-10-13', '2017-10-16', '2017-10-17', '2017-10-18',
       '2017-10-19', '2017-10-20', '2017-10-23', '2017-10-24',
       '2017-10-25', '2017-10-26', '2017-10-27', '2017-10-30',
       '2017-10-31', '2017-11-01', '2017-11-02', '2017-11-03',
       '2017-11-06', '2017-11-07', '2017-11-08', '2017-11-09',
       '2017-11-10', '2017-11-13', '2017-11-14', '2017-11-15',
       '2017-11-16', '2017-11-17', '2017-11-20', '2017-11-21',
       '2017-11-22', '2017-11-24', '2017-11-27', '2017-11-28',
       '2017-11-29', '2017-11-30', '2017-12-01', '2017-12-04',
       '2017-12-05', '2017-12-06', '2017-12-07', '2017-12-08',
       '2017-12-11', '2017-12-12', '2017-12-13', '2017-12-14',
       '2017-12-15', '2017-12-18', '2017-12-19', '2017-12-20',
       '2017-12-21', '2017-12-22', '2017-12-26', '2017-12-27',
       '2017-12-28', '2017-12-29', '2018-01-02', '2018-01-03',
       '2018-01-04', '2018-01-05'], dtype=object)